In [72]:
import yfinance as yf
import pandas as pd
import math

In [73]:
data = yf.download('BTC-USD', start='2020-01-01')
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,7200.174316,18565664997
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,6985.470215,20802083465
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,7344.884277,28111481032
2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,7410.656738,18444271275
2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,7411.317383,19725074095
...,...,...,...,...,...,...
2024-05-23,69121.304688,70041.273438,66356.953125,67929.562500,67929.562500,41895680979
2024-05-24,67928.132812,69220.296875,66622.671875,68526.101562,68526.101562,29197308153
2024-05-25,68526.921875,69579.320312,68515.820312,69265.945312,69265.945312,15473071741


In [76]:
def man_ema_cross_2l(df, fast=12, slow=26):
    #index(Datetime) columns = [['Open','High','Low','Close','']]
    df = data.copy()
    
    #Initital Indicator
    ema_fast = df.Close.ewm(span=fast, adjust=False, min_periods=fast).mean()
    ema_slow = df.Close.ewm(span=slow, adjust=False, min_periods=slow).mean()
    
    #Add Indicator to DataFrame
    df['ema_fast'] = ema_fast
    df['ema_slow'] = ema_slow
    
    #Create Action buy & sell
    df['trend'] = df['ema_fast'] > df['ema_slow']
    df.loc[(df['trend'] == True) & (df['trend'].shift() == False), 'action'] = 'buy'
    df.loc[(df['trend'] == False) & (df['trend'].shift() == True), 'action'] = 'sell'
    
    #Create Action Price
    df['action_price'] = df['Open'].shift(-1)
    
    #Create New DataFrame Not NaN
    trade_table = df[df['action'].notnull()]
    
    #Return
    trade_table['return'] = trade_table['action_price'].pct_change()
    long_position = trade_table[(trade_table['action'] == 'sell') & (trade_table['return'].notnull())]
    short_position = trade_table[(trade_table['action'] == 'buy') & (trade_table['return'].notnull())]
    short_position = short_position['return'] * -1
    
    #Geometric Return Formula
    geo_long = math.prod((1+long_position['return']))**(1/len(long_position)) -1
    geo_short = math.prod((1+short_position))**(1/len(short_position)) -1

    print(f"Long Strategy Return : {geo_long :.2%}")
    print(f"Short Strategy Return : {geo_short :.2%}")

In [77]:
man_ema_cross_2l(data)

Long Strategy Return : 11.69%
Short Strategy Return : -0.99%


C:\Users\ASUS\AppData\Local\Temp\ipykernel_16804\2481907580.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trade_table['return'] = trade_table['action_price'].pct_change()
